# Import functions

In [163]:
import os
import json
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from typing import List, Dict, Tuple
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image, Spacer
from reportlab.lib.units import inch
import tempfile

# Function Definitions

## Loading JSON

In [164]:
def load_json_files(folder_path: str) -> Dict[str, List[dict]]:
    """Load JSON files from a folder and group data by tool."""
    data_by_tool = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            with open(os.path.join(folder_path, file_name), 'r') as f:
                try:
                    data = json.load(f)
                    if isinstance(data, list) and data:
                        tool_name = data[0].get('toolName', 'Unknown')
                        data_by_tool.setdefault(tool_name, []).extend(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding {file_name}: {e}")
    return data_by_tool

## Extracting Metrics from JSON

In [165]:
def extract_metrics(data_by_tool: Dict[str, List[dict]]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Extract metrics from nested JSON data.
    Returns three DataFrames: file transfer successes, file transfer failures, and web test metrics.
    """
    success_records, failure_records, webtest_records = [], [], []
    
    for tool_name, measurements in data_by_tool.items():
        for measurement in measurements:
            # Process file transfers
            for transfer in measurement.get('fileTransfers', []):
                record = {
                    'tool_name': tool_name,  # Tool name from JSON
                    'measurement_number': measurement['measurementNumber'],
                    'transferSuccess': transfer['transferSuccess'],
                    'filename': transfer['filename'],
                    'file_size': transfer['fileSize'],
                    'throughput': transfer['downloadSpeed'] * 8 / 1_000_000,  # Convert to Mbps
                    'dns_lookup': transfer['dnsLookup'] / 1000,  # seconds
                    'tcp_connection': transfer['tcpConnection'] / 1000,
                    'tls_handshake': transfer['tlsHandshake'] / 1000,
                    'ttfb': transfer['timeToFirstByte'] / 1000,
                    'total_time': transfer['totalTransferTime'] / 1000,
                    'transfer_success': transfer['transferSuccess'],
                    'hashMatch': transfer.get('hashMatch', False),
                    'sizeMatch': transfer.get('sizeMatch', False),
                    'percentage_downloaded': transfer.get('percentDownloaded', None) / 100 if transfer.get('percentDownloaded') is not None else None
                }
                if transfer['transferSuccess']:
                    success_records.append(record)
                else:
                    failure_records.append(record)
            
            # Process web tests
            for test in measurement.get('webTests', []):
                record = {
                    'tool_name': tool_name,
                    'measurement_number': measurement['measurementNumber'],
                    'url': test.get('url', ''),
                    'statusCode': test.get('statusCode'),
                    'downloadSpeed': test.get('downloadSpeed')/1000,
                    'uploadSpeed': test.get('uploadSpeed'),
                    'dnsLookup': test.get('dnsLookup'),
                    'tcpConnection': test.get('tcpConnection'),
                    'tlsHandshake': test.get('tlsHandshake'),
                    'timeToFirstByte': test.get('timeToFirstByte'),
                    'totalTime': test.get('totalTime'),
                    'fcp': test.get('fcp'),
                    'lcp': test.get('lcp'),
                    'speedIndex': test.get('speedIndex'),
                    'curlTotalTime': test.get('curlTotalTime'),
                    'playwrightTotalTime': test.get('playwrightTotalTime'),
                    'error': test.get('error')
                }
                webtest_records.append(record)
                
    return pd.DataFrame(success_records), pd.DataFrame(failure_records), pd.DataFrame(webtest_records)



In [166]:
def filter_by_success_rate(success_df: pd.DataFrame, failure_df: pd.DataFrame, threshold: float = 0.75) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Filter tools based on true success rate (hashMatch and sizeMatch both True).
    Tools with no true successes (e.g., 100% partials) are moved to failures.
    
    Parameters:
    - success_df: DataFrame of successful transfers
    - failure_df: DataFrame of failed transfers
    - threshold: Success rate threshold for true successes (default 0.75)
    
    Returns:
    - Tuple of filtered success_df (true successes ≥ threshold) and updated failure_df
    """
    # Define true success: transferSuccess, hashMatch, and sizeMatch all True
    true_success_df = success_df[
        (success_df['transferSuccess'] == True) &
        (success_df['hashMatch'] == True) &
        (success_df['sizeMatch'] == True)
    ]
    
    # Count true successes per tool
    true_success_counts = true_success_df.groupby('tool_name').size().reset_index(name='true_success_count')
    
    # Count all transfers (success + failure) per tool
    success_counts = success_df.groupby('tool_name').size().reset_index(name='success_count')
    failure_counts = failure_df.groupby('tool_name').size().reset_index(name='failure_count')
    total_counts = success_counts.merge(failure_counts, on='tool_name', how='outer').fillna(0)
    total_counts['total'] = total_counts['success_count'] + total_counts['failure_count']
    
    # Merge true success counts with total counts
    total_counts = total_counts.merge(true_success_counts, on='tool_name', how='left').fillna({'true_success_count': 0})
    
    # Calculate true success rate
    total_counts['true_success_rate'] = total_counts['true_success_count'] / total_counts['total']
    
    # Identify tools with true success rate >= threshold and at least one true success
    high_success_tools = total_counts[
        (total_counts['true_success_rate'] >= threshold) & 
        (total_counts['true_success_count'] > 0)
    ]['tool_name']
    
    # Identify tools with no true successes or below threshold
    low_success_tools = total_counts[
        (total_counts['true_success_rate'] < threshold) | 
        (total_counts['true_success_count'] == 0)
    ]['tool_name']
    
    # Filter success_df to keep only high-success tools
    filtered_success_df = success_df[success_df['tool_name'].isin(high_success_tools)].copy()
    
    # Move low-success tools (including 100% partials) from success_df to failure_df
    low_success_df = success_df[success_df['tool_name'].isin(low_success_tools)].copy()
    updated_failure_df = pd.concat([failure_df, low_success_df]).reset_index(drop=True)
    
    print(f"Tools with true success rate ≥{threshold*100}% and at least one true success: {list(high_success_tools)}")
    print(f"Tools with true success rate <{threshold*100}% or no true successes (moved to failures): {list(low_success_tools)}")
    
    return filtered_success_df, updated_failure_df

## Confidence Interval

In [167]:
def calculate_confidence_interval(data: np.array, confidence=0.95) -> Tuple[float, float]:
    """Calculate confidence interval for the mean."""
    mean, sem = np.mean(data), stats.sem(data)
    ci = stats.t.interval(confidence, len(data)-1, loc=mean, scale=sem)
    return ci[0], ci[1]

## Graphing Functions

### Success - Partial - Failure Graphing

In [168]:
import numpy as np

def get_log_ticks(data):
    """Compute major and minor tick values and labels for logarithmic scales, ensuring all major ticks are present and y-axis origin is floored to the nearest major tick."""
    if len(data) == 0:
        return [0.1, 1, 10, 100, 1000], ['0.1', '1', '10', '100', '1000'], [0.2, 0.5, 2, 5, 20, 50, 200, 500]
    
    filtered_data = [x for x in data if x > 0 and np.isfinite(x)]
    if not filtered_data:
        return [0.1, 1, 10, 100, 1000], ['0.1', '1', '10', '100', '1000'], [0.2, 0.5, 2, 5, 20, 50, 200, 500]
    
    min_val = min(filtered_data)
    max_val = max(filtered_data)
    
    try:
        # Floor the minimum value to the nearest major tick (power of 10)
        min_pow = int(np.floor(np.log10(min_val)))
        max_pow = int(np.ceil(np.log10(max_val)))
        
        # Ensure all major ticks are included up to a reasonable limit
        if max_pow - min_pow > 10:  # Cap at 10 orders of magnitude to prevent excessive ticks
            max_pow = min_pow + 10
        
        # Major ticks: Include all powers of 10 from min_pow to max_pow
        major_ticks = [10**i for i in range(min_pow, max_pow + 1)]
        tick_labels = [str(10**i) for i in range(min_pow, max_pow + 1)]
        
        # Minor ticks: Include all integers from 2 to 9 for each power of 10 within the range
        minor_ticks = []
        for i in range(min_pow, max_pow + 1):
            base = 10 ** i
            minor_ticks.extend([base * j for j in range(2, 10) if base * j <= max_val])
        # Remove minor ticks below min_val
        minor_ticks = [t for t in minor_ticks if t >= min_val]
        
        return major_ticks, tick_labels, minor_ticks
    
    except (OverflowError, ValueError):
        return [0.1, 1, 10, 100, 1000], ['0.1', '1', '10', '100', '1000'], [0.2, 0.5, 2, 5, 20, 50, 200, 500]

def apply_common_layout(fig, x_title, y_title, log_y=False, data_for_ticks=None, additional_layout=None):
    """Apply consistent styling to a Plotly figure."""
    layout = dict(
        template='plotly_white',
        font=dict(family='Arial', size=22, color='black', weight='bold'),
        title_font=dict(family='Arial', size=24, color='black', weight='bold'),
        xaxis=dict(
            showline=True,
            linecolor='black',
            linewidth=4,
            mirror=True,
            title=x_title,
            ticks='outside',
            ticklen=8,
            tickwidth=1.5,
            tickcolor='black',
            tickfont=dict(family='Arial', size=22, weight='bold'),
            showgrid=False,
            zeroline=False
        ),
        yaxis=dict(
            showline=True,
            linecolor='black',
            linewidth=4,
            mirror=True,
            title=y_title,
            gridcolor='black',
            gridwidth=0.5,
            ticks='outside',
            ticklen=8,
            tickwidth=1.5,
            tickcolor='black',
            tickfont=dict(family='Arial', size=22, weight='bold'),
            zeroline=False,
            type='log' if log_y else 'linear'
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=80, r=30, t=50, b=80),
        height=700,
        width=1200
    )
    if additional_layout:
        layout.update(additional_layout)
    
    fig.update_layout(**layout)
    
    if log_y and data_for_ticks is not None:
        major_ticks, tick_labels, minor_ticks = get_log_ticks(data_for_ticks)
        fig.update_yaxes(
            tickvals=major_ticks,
            ticktext=tick_labels,
            minor=dict(
                tickvals=minor_ticks,
                ticklen=4,
                tickwidth=1,
                tickcolor='black',
                showgrid=False,  # Enable minor grid lines for better visibility
                gridcolor='lightgray',
                gridwidth=0.3
            )
        )
    
    return fig

### Toolwise Line Graphing

In [169]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

def plot_ttfb_cdf(web_df: pd.DataFrame):
    """
    Creates a CDF plot for Time to First Byte (TTFB) of web tests across different tools.

    Parameters:
    -----------
    web_df : pandas.DataFrame
        DataFrame containing web test metrics, including 'tool_name' and 'timeToFirstByte' (in ms).
    """
    # Extract and prepare TTFB data for each tool, converting ms to seconds
    ttfbs = {}
    min_positive = np.inf
    max_ttfb = 0
    for tool in sorted(web_df['tool_name'].unique()):
        tool_data = web_df[web_df['tool_name'] == tool]['timeToFirstByte'].dropna() / 1000  # Convert ms to s
        tool_data = tool_data[(tool_data > 0) & (np.isfinite(tool_data))]  # Filter invalid
        if not tool_data.empty:
            tool_data_s = tool_data  # Store in seconds
            min_positive = min(min_positive, tool_data_s.min())
            ttfbs[tool] = np.sort(tool_data_s)
            max_ttfb = max(max_ttfb, max(ttfbs[tool]))

    if not ttfbs:
        print("No valid TTFB data available for web tests to plot.")
        return

    if not np.isfinite(min_positive):
        min_positive = 0.01  # 10 ms = 0.01 s

    # Initialize the figure
    fig = go.Figure()

    # Add CDF traces for each tool
    colors = px.colors.qualitative.Safe
    for idx, (tool, data) in enumerate(ttfbs.items()):
        x = np.clip(data, min_positive, None)  # Prevent 0 values
        y = np.arange(1, len(data) + 1) / len(data)
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines+markers',
            name=tool,
            line=dict(color=colors[idx % len(colors)], width=1.5),
            marker=dict(size=4, opacity=0.7),
            showlegend=True
        ))
    
    # Apply layout
    fig.update_layout(
        title='',
        plot_bgcolor='white',    # <-- White background
        paper_bgcolor='white',   # <-- White full figure background
        template='plotly_white',
        font=dict(family='Arial', size=20, color='black', weight='bold'),
        title_font=dict(family='Arial', size=23, color='black', weight='bold'),
        xaxis=dict(
            title='Time to First Byte (s)',
            type='log',
            range=[np.log10(min_positive) - 0.5, np.log10(max_ttfb) + 0.2],
            showline=True,
            mirror=True,
            linecolor='black',
            linewidth=1.5,
            ticks='outside',
            ticklen=8,
            tickwidth=1.5,
            tickcolor='black',
            tickfont=dict(family="Arial", size=20, color='black', weight='bold'),
            showgrid=True,
            gridcolor='#E0E0E0',
            tickangle=45,
            gridwidth=0.5,
            zeroline=False,
            tickformat=".2f",  
        ),
        yaxis=dict(
            title='ECDF',
            range=[0, 1],
            tickmode='array',
            tickvals=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
            showline=True,
            mirror=True,
            linecolor='black',
            linewidth=1.5,
            ticks='outside',
            ticklen=8,
            tickwidth=1.5,
            tickcolor='black',
            tickfont=dict(family="Arial", size=20, color='black', weight='bold'),
            showgrid=True,
            gridcolor='#E0E0E0',
            gridwidth=0.5
        ),
        legend=dict(
            title='Tool',
            yanchor='top',
            y=1,
            xanchor='left',
            x=1.02,
            font=dict(family='Arial', size=18, color='black', weight='bold'),
            bgcolor='rgba(255, 255, 255, 0.5)',
            bordercolor='black',
            borderwidth=1
        ),
        margin={'r': 150},
        showlegend=True
    )

    # Display the plot
    fig.show()

# Data Analysis

### % Download

In [170]:
# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# from typing import List

def plot_web_test_metrics(web_df: pd.DataFrame):
    """
    Plots box plots for various web test metrics across different tools on a log scale.
    Excludes 'onion' tools from TLS and DNS graphs.

    Parameters:
    -----------
    web_df : pandas.DataFrame
        DataFrame containing web test metrics, including 'tool_name' and various metrics.
    """
    metrics = ['dnsLookup', 'tcpConnection', 'tlsHandshake', 'timeToFirstByte', 'totalTime', 
               'downloadSpeed', 'playwrightTotalTime', 'curlTotalTime']
    font_family = "Arial"
    label_font_size = 24
    title_font_size = 24
    tick_font_size = 22
    line_width = 2.5
    marker_size = 5
    
    color_palette = [
        '#1f77b4',  # Muted Blue
        '#ff7f0e',  # Vivid Orange
        '#2ca02c',  # Rich Green
        '#d62728',  # Clear Red
        '#9467bd',  # Elegant Purple
        '#8c564b',  # Earthy Brown
        '#e377c2',  # Soft Pink
        '#7f7f7f',  # Neutral Gray
        '#bcbd22',  # Yellow-Green
        '#17becf',  # Aqua Cyan
        '#aec7e8',  # Light Blue
        '#ffbb78',  # Light Orange
        '#98df8a',  # Light Green
        '#c5b0d5'   # Light Purple
    ]


    
    # Define tool groups, specifically to exclude 'onion' tools
    tool_groups = {
        'https': ['Cloudflared', 'Zrok', 'Loophole', 'Telebit', 'Ngrok', 'Beeceptor', 'Tunnel.Pyjam.as', 'LocalhostRun'],
        'http': ['Bore', 'Localxpose'],
        'ssh': ['Serveo', 'Pinggy'],
        'onion': ['Ngtor', 'Onionpipe', 'EphemeralHiddenService', 'EphemeralHS']
    }
    onion_tools = tool_groups['onion']
    
    for metric in metrics:
        fig = go.Figure()
        all_metric_data = []
        # Filter out 'onion' tools for tlsHandshake and dnsLookup
        if metric in ['tlsHandshake', 'dnsLookup', 'tcpConnection']:
            tools_to_plot = [tool for tool in web_df['tool_name'].unique() if tool not in onion_tools]
        else:
            tools_to_plot = web_df['tool_name'].unique()
        
        for i, tool in enumerate(tools_to_plot):
            # Extract and convert data to numeric, coercing errors to NaN
            tool_data = web_df[web_df['tool_name'] == tool][metric].dropna()
            tool_data = pd.to_numeric(tool_data, errors='coerce')  # Convert to numeric, handle non-numeric as NaN
            tool_data = tool_data[(tool_data > 0) & (pd.notna(tool_data))]  # Filter positive and finite values
            if not tool_data.empty:
                all_metric_data.extend(tool_data)
                fig.add_trace(go.Violin(
                    y=tool_data,
                    name=tool,
                    meanline_visible=True,
                    line=dict(width=line_width),
                    opacity=0.7,
                    showlegend=False,
                    fillcolor=color_palette[i % len(color_palette)],
                    line_color='black'
                ))
        
        major_ticks, tick_labels, minor_ticks = get_log_ticks(all_metric_data)
        fig.update_layout(

            yaxis=dict(
                type='log',
                title=dict(
                    text=f"Time (ms)" if metric in ['dnsLookup', 'tlsHandshake', 'tcpConnection'] 
                         else f"(Mbps)" if metric == 'downloadSpeed' else metric,
                    font=dict(family=font_family, size=label_font_size, color='black', weight='bold'),
                    standoff=15
                ),
                tickvals=major_ticks,
                ticktext=tick_labels,
                minor=dict(
                    tickvals=minor_ticks,
                    ticklen=4,
                    tickwidth=1,
                    tickcolor='black',
                    showgrid=False
                ),
                showgrid=True,
                gridcolor='#E0E0E0',
                gridwidth=0.5,
                zeroline=False,
                showline=True,
                linecolor='black',
                linewidth=4,
                mirror=True,
                ticks='outside',
                ticklen=8,
                tickwidth=1.5,
                tickcolor='black',
                tickfont=dict(family=font_family, size=tick_font_size, color='black', weight='bold'),
                exponentformat='e'
            ),
            xaxis=dict(
                showline=True,
                linecolor='black',
                linewidth=4,
                mirror=True,
                ticks='outside',
                ticklen=8,
                tickwidth=1.5,
                tickcolor='black',
                tickfont=dict(family=font_family, size=tick_font_size, color='black', weight='bold'),
                showgrid=False,
                zeroline=False,
                categoryorder='trace'
            ),
            plot_bgcolor='white',
            paper_bgcolor='white',
            margin=dict(l=80, r=30, t=50, b=80),
            height=700,
            width=900,
            showlegend=False
        )
        fig.show()


In [171]:
import pandas as pd
import plotly.express as px

def plot_median_time_by_tool(df: pd.DataFrame):
    """
    Plots a line graph with tools on the x-axis, median total time on the y-axis,
    and different lines for each file size.
    
    Parameters:
    - df: DataFrame containing file transfer data (e.g., success_df).
    """
    # Calculate median total_time for each tool and file size
    plot_df = df.groupby(['tool_name', 'file_size'])['total_time'].median().reset_index(name='median_time')
    
    # Convert file_size to MB
    plot_df['file_size_mb'] = plot_df['file_size'] / (1024 * 1024)
    
    # Create a string representation for file size
    plot_df['file_size_str'] = plot_df['file_size_mb'].apply(lambda x: f"{x:.2f} MB")
    
    # Sort tools alphabetically
    plot_df = plot_df.sort_values(by='tool_name')
    
    # Create line plot
    fig = px.line(
        plot_df,
        x='tool_name',
        y='median_time',
        color='file_size_str',
        title='Median Total Download Time by Tool for Each File Size',
        labels={
            'tool_name': 'Tool',
            'median_time': 'Median Total Time (s)',
            'file_size_str': 'File Size'
        },
        markers=True  # Add markers to show data points
    )
    
    # Get data for log ticks
    data_for_ticks = plot_df['median_time'].dropna()
    
    # Apply common layout (assumed to be defined elsewhere)
    fig = apply_common_layout(
        fig,
        x_title='Tool',
        y_title='Median Total Time (s)',
        log_y=True,
        data_for_ticks=data_for_ticks,
        additional_layout={'legend_title_text': 'File Size'}
    )
    
    # Add vertical gridlines at each x-tick (tool name)
    fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray')

    # Display the plot
    fig.show()


In [172]:
import pandas as pd
import plotly.graph_objects as go

def plot_speed_index(web_df: pd.DataFrame):
    """
    Plots a box plot for the Speed Index metric across different tools on a log scale.

    Parameters:
    -----------
    web_df : pandas.DataFrame
        DataFrame containing web test metrics, including 'tool_name' and 'speedIndex'.
    """
    metric = 'speedIndex'
    fig = go.Figure()
    all_metric_data = []
    
    # Iterate over unique tools and extract Speed Index data
    for tool in web_df['tool_name'].unique():
        tool_data = web_df[web_df['tool_name'] == tool][metric].dropna()
        tool_data = pd.to_numeric(tool_data, errors='coerce')  # Convert to numeric, handle errors
        tool_data = tool_data[(tool_data > 0) & (pd.notna(tool_data))]  # Filter valid positive values
        if not tool_data.empty:
            all_metric_data.extend(tool_data)
            fig.add_trace(go.Box(
                y=tool_data,
                name=tool,
                boxmean=True,
                line=dict(width=1.5),
                opacity=0.7,
                whiskerwidth=0.5,
                boxpoints='outliers',
                jitter=0.3,
                marker=dict(size=5, opacity=0.7, line=dict(width=1)),
                showlegend=True
            ))
    
    # Check if there’s valid data to plot
    if not all_metric_data:
        print("No valid data available for Speed Index.")
        return
    
    # Calculate log scale ticks
    major_ticks, tick_labels, minor_ticks = get_log_ticks(all_metric_data)
    
    # Update figure layout
    fig.update_layout(
        title=dict(
            text="Web Test Metric: Speed Index",
            font=dict(family="Arial", size=20, color='black'),
            x=0.5
        ),
        yaxis=dict(
            type='log',
            title=dict(
                text="Speed Index (ms)",
                font=dict(family="Arial", size=18, color='black'),
                standoff=15
            ),
            tickvals=major_ticks,
            ticktext=tick_labels,
            minor=dict(
                tickvals=minor_ticks,
                ticklen=4,
                tickwidth=1,
                tickcolor='black',
                showgrid=False
            ),
            showgrid=True,
            gridcolor='#E0E0E0',
            gridwidth=0.5,
            zeroline=False,
            showline=True,
            linecolor='black',
            linewidth=1.5,
            mirror=True,
            ticks='outside',
            ticklen=8,
            tickwidth=1.5,
            tickcolor='black',
            tickfont=dict(family="Arial", size=18, color='black'),
            exponentformat='e'
        ),
        xaxis=dict(
            title='Tool',
            showline=True,
            linecolor='black',
            linewidth=1.5,
            mirror=True,
            ticks='outside',
            ticklen=8,
            tickwidth=1.5,
            tickcolor='black',
            tickfont=dict(family="Arial", size=18, color='black', weight='bold'),
            showgrid=False,
            zeroline=False,
            categoryorder='trace'
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=80, r=30, t=50, b=80),
        height=700,
        width=900,
        showlegend=True
    )
    
    # Display the plot
    fig.show()

In [173]:
# def plot_curl_playwright_single_graph(web_df: pd.DataFrame):
#     """
#     Generate a single graph comparing curlTotalTime and playwrightTotalTime for each tool.
#     """
#     # Ensure required columns are present
#     required_columns = ['tool_name', 'measurement_number', 'curlTotalTime', 'playwrightTotalTime']
#     if not all(col in web_df.columns for col in required_columns):
#         print("Required columns are missing from web_df.")
#         return

#     # Filter out invalid data and convert times to seconds
#     plot_df = web_df[required_columns].copy()
#     plot_df = plot_df[(plot_df['curlTotalTime'] > 0) & (plot_df['playwrightTotalTime'] > 0)].dropna()
#     plot_df['curlTotalTime'] = plot_df['curlTotalTime'] / 1000  # Convert ms to s
#     plot_df['playwrightTotalTime'] = plot_df['playwrightTotalTime'] / 1000  # Convert ms to s

#     if plot_df.empty:
#         print("No valid data available for plotting after filtering.")
#         return

#     # Melt the DataFrame for easier plotting
#     melted_df = pd.melt(
#         plot_df,
#         id_vars=['tool_name', 'measurement_number'],
#         value_vars=['curlTotalTime', 'playwrightTotalTime'],
#         var_name='test_type',
#         value_name='total_time'
#     )
#     melted_df['test_type'] = melted_df['test_type'].map({
#         'curlTotalTime': 'Curl',
#         'playwrightTotalTime': 'Playwright'
#     })

#     # Create box plot
#     fig = px.box(
#         melted_df,
#         x='tool_name',
#         y='total_time',
#         color='test_type',
#         title='Comparison of Curl and Playwright Total Time by Tool (Log Scale)',
#         labels={'total_time': 'Total Time (s)', 'tool_name': 'Tool', 'test_type': 'Test Type'}
#     )

#     # Get data for log ticks
#     data_for_ticks = melted_df['total_time'].dropna()

#     # Apply common layout
#     fig = apply_common_layout(
#         fig,
#         x_title='Tool',
#         y_title='Total Time (s)',
#         log_y=True,
#         data_for_ticks=data_for_ticks,
#         additional_layout={'legend_title_text': 'Test Type'}
#     )

#     fig.show()

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def plot_curl_playwright_single_graph(web_df: pd.DataFrame):
    """
    Generate a single graph comparing curlTotalTime and playwrightTotalTime for each tool.
    """
    # Ensure required columns are present
    required_columns = ['tool_name', 'measurement_number', 'curlTotalTime', 'playwrightTotalTime']
    if not all(col in web_df.columns for col in required_columns):
        print("Required columns are missing from web_df.")
        return

    # Filter out invalid data and convert times to seconds
    plot_df = web_df[required_columns].copy()
    plot_df = plot_df[(plot_df['curlTotalTime'] > 0) & (plot_df['playwrightTotalTime'] > 0)].dropna()
    plot_df['curlTotalTime'] = plot_df['curlTotalTime'] / 1000  # Convert ms to s
    plot_df['playwrightTotalTime'] = plot_df['playwrightTotalTime'] / 1000  # Convert ms to s

    if plot_df.empty:
        print("No valid data available for plotting after filtering.")
        return

    # Melt the DataFrame for easier plotting
    melted_df = pd.melt(
        plot_df,
        id_vars=['tool_name', 'measurement_number'],
        value_vars=['curlTotalTime', 'playwrightTotalTime'],
        var_name='test_type',
        value_name='total_time'
    )
    melted_df['test_type'] = melted_df['test_type'].map({
        'curlTotalTime': 'Curl',
        'playwrightTotalTime': 'Playwright'
    })

    # Create figure
    fig = go.Figure()

    # Define colors and line widths
    colors = {'Curl': '#1f77b4', 'Playwright': '#ff7f0e'}  # Blue for Curl, Orange for Playwright
    line_widths = {'Curl': 1, 'Playwright': 4}  # Thicker boundary for Playwright

    # Get unique tools and test types
    tools = melted_df['tool_name'].unique()
    test_types = ['Curl', 'Playwright']

    # Add box plots for each tool and test type
    for tool_idx, tool in enumerate(tools):
        for test_type in test_types:
            # Filter data for the current tool and test type
            data = melted_df[(melted_df['tool_name'] == tool) & (melted_df['test_type'] == test_type)]['total_time']
            if not data.empty:
                # Calculate x position with slight offset: -0.2 for Curl, +0.2 for Playwright
                x_offset = -0.2 if test_type == 'Curl' else 0.2
                x_positions = [tool_idx + x_offset] * len(data)
                fig.add_trace(go.Box(
                    x=x_positions,
                    y=data,
                    name=test_type,
                    line=dict(color='black', width=line_widths[test_type]),
                    fillcolor=colors[test_type],
                    opacity=0.7,
                    showlegend=(tool == tools[0]),  # Show legend only for the first tool
                    legendgroup=test_type
                ))

    # Update layout to match px.box style
    fig.update_layout(
        title='Comparison of Curl and Playwright Total Time by Tool (Log Scale)',
        xaxis_title='Tool',
        yaxis_title='Total Time (s)',
        legend_title_text='Test Type',
        template='plotly_white',
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(tools))),
            ticktext=tools
        )
    )

    # Get data for log ticks
    data_for_ticks = melted_df['total_time'].dropna()

    # Apply common layout
    fig = apply_common_layout(
        fig,
        x_title='Tool',
        y_title='Total Time (s)',
        log_y=True,
        data_for_ticks=data_for_ticks,
        additional_layout={'legend_title_text': 'Test Type'}
    )

    fig.show()

In [174]:
def plot_curl_playwright_separate_graphs(web_df: pd.DataFrame):
    """
    Generate separate graphs for curlTotalTime and playwrightTotalTime for each tool.
    """
    # Ensure required columns are present
    required_columns = ['tool_name', 'measurement_number', 'curlTotalTime', 'playwrightTotalTime']
    if not all(col in web_df.columns for col in required_columns):
        print("Required columns are missing from web_df.")
        return

    # Filter out invalid data and convert times to seconds
    plot_df = web_df[required_columns].copy()
    plot_df = plot_df[(plot_df['curlTotalTime'] > 0) & (plot_df['playwrightTotalTime'] > 0)].dropna()
    plot_df['curlTotalTime'] = plot_df['curlTotalTime'] / 1000  # Convert ms to s
    plot_df['playwrightTotalTime'] = plot_df['playwrightTotalTime'] / 1000  # Convert ms to s

    if plot_df.empty:
        print("No valid data available for plotting after filtering.")
        return

    # Create separate box plots
    # Curl graph
    fig_curl = px.box(
        plot_df,
        x='tool_name',
        y='curlTotalTime',
        title='Curl Total Time by Tool (Log Scale)',
        labels={'curlTotalTime': 'Total Time (s)', 'tool_name': 'Tool'}
    )
    data_for_ticks_curl = plot_df['curlTotalTime'].dropna()
    fig_curl = apply_common_layout(
        fig_curl,
        x_title='Tool',
        y_title='Total Time (s)',
        log_y=True,
        data_for_ticks=data_for_ticks_curl
    )
    fig_curl.show()

    # Playwright graph
    fig_playwright = px.box(
        plot_df,
        x='tool_name',
        y='playwrightTotalTime',
        title='Playwright Total Time by Tool (Log Scale)',
        labels={'playwrightTotalTime': 'Total Time (s)', 'tool_name': 'Tool'}
    )
    data_for_ticks_playwright = plot_df['playwrightTotalTime'].dropna()
    fig_playwright = apply_common_layout(
        fig_playwright,
        x_title='Tool',
        y_title='Total Time (s)',
        log_y=True,
        data_for_ticks=data_for_ticks_playwright
    )
    fig_playwright.show()

In [175]:
import pandas as pd
import plotly.express as px

def create_webtest_sectional_bar_graph(web_df: pd.DataFrame, folder_path: str):
    """
    Creates a sectional bar graph showing the count of successful, failed, and partial web tests by tool.

    Parameters:
    -----------
    web_df : pandas.DataFrame
        DataFrame containing web test metrics, including 'tool_name', 'statusCode', and 'error'.
    folder_path : str
        Path to the folder containing JSON files (for consistency with other functions).
    """
    # Create a copy of the DataFrame to avoid modifying the original
    web_df = web_df.copy()

    # Define success, failure, and partial criteria
    web_df['status'] = 'Failure'  # Default to failure
    # Success: statusCode is 200 and no error
    web_df.loc[(web_df['statusCode'] == 200) & (web_df['error'].isna()), 'status'] = 'Success'
    # Partial: statusCode is 200 but has missing key metrics (e.g., fcp or lcp)
    web_df.loc[(web_df['statusCode'] == 200) & (web_df['error'].isna()) & 
               (web_df['fcp'].isna() | web_df['lcp'].isna()), 'status'] = 'Partial'

    # Count the number of each status per tool
    status_counts = web_df.groupby(['tool_name', 'status']).size().reset_index(name='count')
    
    # Pivot the data to create a table suitable for a stacked bar graph
    status_data = status_counts.pivot_table(
        index='tool_name', 
        columns='status', 
        values='count', 
        aggfunc='sum'
    ).reset_index().fillna(0)

    # Ensure all status columns are present, even if zero
    status_data = status_data.reindex(columns=['tool_name', 'Success', 'Failure', 'Partial'], fill_value=0)

    # Create the bar graph
    fig = px.bar(
        status_data,
        x='tool_name',
        y=['Success', 'Failure', 'Partial'],
        title='Web Test Status by Tool',
        labels={'value': 'Number of Web Tests', 'variable': 'Status', 'tool_name': 'Tool'},
        text_auto=True,
        color_discrete_sequence=px.colors.qualitative.Safe
    )

    # Apply common layout for consistent styling
    fig = apply_common_layout(
        fig,
        x_title='Tool',
        y_title='Number of Web Tests',
        log_y=False,
        additional_layout={'legend_title_text': 'Status', 'bargap': 0.2}
    )

    # Display the plot
    fig.show()

In [176]:
def plot_curl_playwright_with_connected_medians(web_df: pd.DataFrame):
    """
    Create a graph with box plots for Curl and Playwright total times by tool,
    connecting the medians from one tool to the next.
    """
    # Dynamically determine tool order from the DataFrame
    tool_order = sorted(web_df['tool_name'].unique())

    # Check for required columns
    required_columns = ['tool_name', 'measurement_number', 'curlTotalTime', 'playwrightTotalTime']
    if not all(col in web_df.columns for col in required_columns):
        print("Error: Missing required columns in the DataFrame.")
        return

    # Filter invalid data and convert times to seconds
    plot_df = web_df[required_columns].copy()
    plot_df = plot_df[(plot_df['curlTotalTime'] > 0) & (plot_df['playwrightTotalTime'] > 0)].dropna()
    plot_df['curlTotalTime'] = plot_df['curlTotalTime'] / 1000  # ms to s
    plot_df['playwrightTotalTime'] = plot_df['playwrightTotalTime'] / 1000  # ms to s

    if plot_df.empty:
        print("Error: No valid data after filtering.")
        return

    # Melt the DataFrame to long format
    melted_df = pd.melt(
        plot_df,
        id_vars=['tool_name', 'measurement_number'],
        value_vars=['curlTotalTime', 'playwrightTotalTime'],
        var_name='test_type',
        value_name='total_time'
    )
    melted_df['test_type'] = melted_df['test_type'].map({
        'curlTotalTime': 'Curl',
        'playwrightTotalTime': 'Playwright'
    })

    # Create box plots
    fig = px.box(
        melted_df,
        x='tool_name',
        y='total_time',
        color='test_type',
        title='Curl vs Playwright Total Time by Tool with Connected Medians',
        labels={'total_time': 'Total Time (s)', 'tool_name': 'Tool', 'test_type': 'Test Type'},
        category_orders={'tool_name': tool_order},
        color_discrete_map={'Curl': '#1f77b4', 'Playwright': '#ff7f0e'}  # Blue for Curl, Orange for Playwright
    )

    # Calculate medians
    medians = melted_df.groupby(['tool_name', 'test_type'])['total_time'].median().reset_index()

    # Add lines connecting medians for each test type
    for test_type, color in [('Curl', '#2ca02c'), ('Playwright', '#9467bd')]:  # Green for Curl median, Purple for Playwright median
        test_medians = medians[medians['test_type'] == test_type]
        # Ensure tool order matches the dynamically determined sequence
        test_medians = test_medians.set_index('tool_name').reindex(tool_order).reset_index()
        fig.add_trace(go.Scatter(
            x=test_medians['tool_name'],
            y=test_medians['total_time'],
            mode='lines+markers',
            name=f'{test_type} Median',
            line=dict(dash='dash', color=color),
            marker=dict(size=8, color=color)
        ))

    # Apply common layout with log scale
    data_for_ticks = melted_df['total_time'].dropna()
    fig = apply_common_layout(
        fig,
        x_title='Tool',
        y_title='Total Time (s)',
        log_y=True,
        data_for_ticks=data_for_ticks,
        additional_layout={'legend_title_text': 'Test Type'}
    )

    # Display the plot
    fig.show()

# Example usage (replace 'web_df' with your DataFrame):
# plot_curl_playwright_with_connected_medians(web_df)

# Load Data

In [177]:
folder_path = "analyse_these"  # Your folder with JSON files
data_by_tool = load_json_files(folder_path)
success_df, failure_df, web_df = extract_metrics(data_by_tool)

## Size is true but HashMatch is false

In [178]:
# filtered_success_df, updated_failure_df = filter_by_success_rate(success_df, failure_df, threshold=0.75)

# # Generate the mismatch table and print summary
# mismatch_table = generate_size_hash_mismatch_table(success_df)
# print(mismatch_table)
# mismatch_table.to_csv("size_hash_mismatch_table.csv", index=False)

# print("Size Match True and Hash Match False Count:", 
#       success_df[(success_df['sizeMatch'] == True) & (success_df['hashMatch'] == False)].shape[0])

# print(success_df.head())
# print(failure_df.head())
# print(success_df.isnull().sum())
# print(failure_df.isnull().sum())
# print(success_df['tool_name'].unique())
# print(success_df['measurement_number'].unique())
# print(failure_df['tool_name'].unique())
# print(failure_df['measurement_number'].unique())

# Plotting Graphs

In [179]:
# Load Data with enhanced handler
folder_path = "analyse_these"
data_by_tool = load_json_files(folder_path)

# Process data if valid JSON was loaded
if data_by_tool:
    success_df, failure_df, web_df = extract_metrics(data_by_tool)
    
    # Generate Web Test Graphs
    if not web_df.empty:
        plot_ttfb_cdf(web_df)
        plot_speed_index(web_df)
        plot_web_test_metrics(web_df)  # Shows FCP/LCP
        plot_curl_playwright_with_connected_medians(web_df)
    else:
        print("No web test data found.")
else:
    print("No valid JSON data found in folder.")